In [1]:
import pandas as pd
import numpy as np

In [2]:
path = r"C:\Users\zjxia\iCloudDrive\Career\FBL_analysis\sample_data\FBL199 Claire Lancaster 4plexE OS 30.5.24.csv"

In [3]:
df =  pd.read_csv(path)

In [4]:
unit = df.Unit.unique()
df = df[['Sample Barcode', 'Assay', 'Plex', 'Location', 'Carrier Barcode',
       'Replicate AEB', 'Mean AEB', 'SD AEB', 'CV AEB', 
        'Replicate Conc.','Mean Conc.', 'SD Conc.', 'CV Conc.',
         #'Unit', 'Job Status', 'Job ID',
       'Flags', 'Errors']]
# Only keep columns listed above

In [5]:
unit[0]

'pg/mL'

In [6]:
# This part is more fragile than others because we are using specific condition to select data entries.
# There should be no uncategorised entry, if there is, check the "Sample Barcode" naming.

# Any entry with "Calibrator" in their "Sample Barcode" is a calibrator
df_cals = df.loc[df['Sample Barcode'].str.contains('Calibrator')] 
# Any entry with "Control" or "Int Ctrl" in their "Sample Barcode" is a control
df_ctrls = df.loc[(df['Sample Barcode'].str.contains('Control'))|(df['Sample Barcode'].str.contains('Int Ctrl'))]
# All entries with numerical "Sample Barcode" is a sample
df_samples = df.loc[df['Sample Barcode'].str.isnumeric()]

leftover = len(df)-len(df_cals)-len(df_ctrls)-len(df_samples)
print(f"{len(df_cals)} calibration levels, {len(df_ctrls)} controls and {len(df_samples)} samples were found. \n{leftover} entries were uncategorised.")

96 calibration levels, 72 controls and 216 samples were found. 
0 entries were uncategorised.


In [30]:
def due_with_replicates(df):
    """
    input
    """
    # Group by Categories and aggregate the values into lists
    grouped_df = df.groupby(['Sample Barcode', 'Assay', 'Plex', 'Location', 'Carrier Barcode']).agg(list).reset_index()

    # lambda x: [value for value in x if not pd.isna(value)] # This function returns all the non-NaN values
    grouped_df['Replicate AEB'] = grouped_df['Replicate AEB'].apply(lambda x: [value for value in x if not pd.isna(value)])
    grouped_df['Replicate Conc.'] = grouped_df['Replicate Conc.'].apply(lambda x: [value for value in x if not pd.isna(value)])

    # Define a function to extract the first non-NaN value from a list
    def first_non_nan(lst):
        for item in lst:
            if not pd.isna(item):
                return item
        return np.nan  # Return NaN if all values are NaN

    # Manually expand the lists into separate columns
    expanded_df = pd.DataFrame()
    expanded_df['Sample Barcode'] = grouped_df['Sample Barcode']
    expanded_df['Assay'] = grouped_df['Assay']
    expanded_df['Plex'] = grouped_df['Plex']
    expanded_df['Location'] = grouped_df['Location']
    expanded_df['Carrier Barcode'] = grouped_df['Carrier Barcode']
    
    # Create new columns for each AEB replicates in the lists
    if grouped_df['Replicate AEB'].apply(len).max() == 1:
        expanded_df[f'Replicate AEB'] = grouped_df['Replicate AEB'].apply(first_non_nan)
    else:
        for i in range(grouped_df['Replicate AEB'].apply(len).max()):  # Get the maximum length of lists in Value1
            expanded_df[f'Replicate AEB {i+1}'] = grouped_df['Replicate AEB'].apply(lambda x: x[i] if len(x) > i else None)

    expanded_df['Mean AEB'] = grouped_df['Mean AEB'].apply(first_non_nan)
    expanded_df['SD AEB'] = grouped_df['SD AEB'].apply(first_non_nan)
    expanded_df['CV AEB'] = grouped_df['CV AEB'].apply(first_non_nan)
    
    # Create new columns for each Conc. replicates in the lists
    if grouped_df['Replicate Conc.'].apply(len).max() == 1:
        expanded_df[f'Replicate Conc. ({unit[0]})'] = grouped_df['Replicate Conc.'].apply(first_non_nan)
    else:
        for i in range(grouped_df['Replicate Conc.'].apply(len).max()):  # Get the maximum length of lists in Value1
            expanded_df[f'Replicate Conc. {i+1}'] = grouped_df['Replicate Conc.'].apply(lambda x: x[i] if len(x) > i else None)
    
    expanded_df[f'Mean Conc. ({unit[0]})'] = grouped_df['Mean Conc.'].apply(first_non_nan)
    expanded_df['SD Conc.'] = grouped_df['SD Conc.'].apply(first_non_nan)
    expanded_df['CV Conc.'] = grouped_df['CV Conc.'].apply(first_non_nan)

    expanded_df['Flags'] = grouped_df['Flags'].apply(first_non_nan)
    expanded_df['Errors'] = grouped_df['Errors'].apply(first_non_nan)
    
    expanded_df = expanded_df.sort_values(by=['Assay', "Plex", 'Sample Barcode'])
    expanded_df = expanded_df.reset_index(drop=True)
    return expanded_df

In [31]:
# Process calibration standards
df_cals = due_with_replicates(df_cals)
# Process controls
df_ctrls = due_with_replicates(df_ctrls)
# Process samples
df_samples = due_with_replicates(df_samples)

In [43]:
# Calculate intra-plate %CV
# Based on Int Ctrls and drop all entries for machine calculated means
df_intra_CV = df.loc[(df['Sample Barcode'].str.contains('Int Ctrl'))&(df['Replicate Conc.'].notna())]
# Calculate for each plex
df_intra_CV = df_intra_CV.groupby(['Assay', 'Plex']).agg(Average=('Replicate Conc.','mean'),
                                                          Stdev=('Replicate Conc.','std'))

In [44]:
df_intra_CV = df_intra_CV.reset_index()
df_intra_CV['%CV'] = df_intra_CV['Stdev']/df_intra_CV['Average']*100

In [45]:
df_intra_CV

,Assay,Plex,Average,Stdev,%CV
0,Neuro 4-Plex E,Abeta 40,95.063798,6.283075,6.609325
1,Neuro 4-Plex E,Abeta 42,5.232007,0.234773,4.487241
2,Neuro 4-Plex E,GFAP,104.913739,11.589510,11.046704
3,Neuro 4-Plex E,NF-light,25.283575,3.140094,12.419502


In [9]:
# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(path.replace('.csv', ' sorted_1.xlsx'), engine='xlsxwriter') as writer:
    # Write each DataFrame to a different worksheet
    df_samples.to_excel(writer, sheet_name='Samples', index=False)
    df_ctrls.to_excel(writer, sheet_name='Controls', index=False)
    df_cals.to_excel(writer, sheet_name='Cals', index=False)
    
    # Access the workbook and worksheet objects
    workbook  = writer.book
    worksheet_cal = writer.sheets['Cals']


    # Define a format for the highlighted cells
    highlight_format = workbook.add_format({'bg_color': 'yellow'})

    # Apply conditional formatting based on criteria
    worksheet1.conditional_format('J2:J1000', {'type': 'cell',
                                            'criteria': '>',
                                            'value': 0.15,
                                            'format': highlight_format})

print("Excel file with highlighted cells created successfully.")

ModuleNotFoundError: No module named 'xlsxwriter'